# 01. データ探索 - 予測結果ファイルの確認

このNotebookでは、予測結果ファイルを読み込んで基本的な情報を確認します。

## 目的
- 予測結果ファイルの構造理解
- データ品質のチェック
- 基本統計量の確認

## 1. ライブラリのインポート

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# 日本語フォント設定
plt.rcParams['font.sans-serif'] = ['MS Gothic', 'Yu Gothic', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# グラフスタイル
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ ライブラリのインポート完了")

## 2. データの読み込み

In [ ]:
# ファイルパスを指定（環境に合わせて変更してください）
data_path = Path("../../check_results/predicted_results_all.tsv")

# TSVファイルの読み込み
df = pd.read_csv(data_path, sep='\t')

print(f"✅ データ読み込み完了")
print(f"📊 総レコード数: {len(df):,}")
print(f"📅 カラム数: {len(df.columns)}")

## 3. データ構造の確認

In [ ]:
# 最初の数行を表示
print("=== データサンプル ===")
df.head()

In [ ]:
# カラム一覧
print("=== カラム一覧 ===")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

In [ ]:
# データ型と欠損値
print("=== データ型と欠損値 ===")
df.info()

## 4. 基本統計量

In [ ]:
# レース数を集計
race_count = df.groupby(['競馬場', '開催年', '開催日', 'レース番号']).size().reset_index(name='出走頭数')

print(f"📍 総レース数: {len(race_count):,}")
print(f"🏇 総出走馬数: {len(df):,}")
print(f"📊 平均出走頭数: {race_count['出走頭数'].mean():.1f}頭")

In [ ]:
# 年別のレース数
print("\n=== 年別レース数 ===")
year_stats = df.groupby('開催年')['レース番号'].count().reset_index(name='馬数')
year_stats['レース数'] = df.groupby('開催年').apply(lambda x: len(x.groupby(['開催日', 'レース番号']))).values
print(year_stats)

In [ ]:
# 競馬場別のレース数
print("\n=== 競馬場別レース数 ===")
track_stats = df.groupby('競馬場').apply(lambda x: len(x.groupby(['開催年', '開催日', 'レース番号']))).sort_values(ascending=False)
print(track_stats)

## 5. 穴馬データの確認

In [ ]:
# 穴馬候補の統計
if '穴馬候補' in df.columns:
    upset_count = df['穴馬候補'].sum()
    upset_hit = df[df['穴馬候補'] == 1]['実際の穴馬'].sum() if '実際の穴馬' in df.columns else 0
    
    print("=== 穴馬予測統計 ===")
    print(f"🎯 穴馬候補数: {upset_count:,}")
    print(f"✅ 穴馬的中数: {upset_hit:,}")
    if upset_count > 0:
        print(f"📈 適合率: {upset_hit/upset_count*100:.2f}%")
else:
    print("⚠️ 穴馬候補カラムが見つかりません")

## 6. オッズ分布の確認

In [ ]:
# オッズのヒストグラム
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 線形スケール
axes[0].hist(df['単勝オッズ'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('単勝オッズ')
axes[0].set_ylabel('頻度')
axes[0].set_title('単勝オッズ分布（線形）')
axes[0].grid(True, alpha=0.3)

# 対数スケール（オッズが大きい馬も見やすく）
axes[1].hist(df['単勝オッズ'], bins=50, edgecolor='black', alpha=0.7)
axes[1].set_xlabel('単勝オッズ')
axes[1].set_ylabel('頻度')
axes[1].set_title('単勝オッズ分布（対数）')
axes[1].set_yscale('log')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. 予測精度の確認

In [ ]:
# 予測順位と実際の着順の相関
from scipy.stats import spearmanr

correlation, p_value = spearmanr(df['予測順位'], df['確定着順'])

print("=== 予測精度 ===")
print(f"📊 スピアマンの順位相関係数: {correlation:.4f}")
print(f"📈 p値: {p_value:.4e}")

if correlation > 0.5:
    print("✅ 強い正の相関！予測モデルは有効です")
elif correlation > 0.3:
    print("✅ 中程度の相関。改善の余地あり")
else:
    print("⚠️ 相関が弱い。モデルの見直しが必要かも")

In [ ]:
# 予測1位の的中率
predicted_1st = df[df['予測順位'] == 1]
win_rate = (predicted_1st['確定着順'] == 1).sum() / len(predicted_1st)

print(f"\n🎯 予測1位の単勝的中率: {win_rate*100:.2f}%")
print(f"📊 予測1位の総数: {len(predicted_1st):,}")
print(f"✅ 的中数: {(predicted_1st['確定着順'] == 1).sum():,}")

## 8. まとめ

このNotebookで確認したこと:
- ✅ データの基本構造
- ✅ レース数・出走馬数の統計
- ✅ 穴馬予測の精度
- ✅ オッズ分布
- ✅ 予測精度の評価

次のステップ:
- 📊 `02_backtest_analysis.ipynb` でバックテスト分析
- 🎲 `03_monte_carlo_analysis.ipynb` でモンテカルロシミュレーション